In [44]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db

In [45]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [46]:
year2_hourly, meta = Db.load_data(hourly=True, meta=True, year=2)

In [47]:
consumption = meta.loc[((meta['Parameter'] == "Power_Electrical") | (meta['Parameter'] == "Power_Thermal")) & (
    meta["Description"].str.contains("power consumption" or "used"))].index.tolist()

In [48]:
year1_consumption = year1_hourly[["Timestamp"]+consumption].copy()

In [49]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
year1_consumption['Timestamp'] = pd.to_datetime(year1_consumption['Timestamp'], errors="coerce", utc=True, format="%Y-%m-%d %H:%M:%S%z")

#Extracts hour of Timestamp.
year1_consumption["HourOfTimestamp"] = year1_consumption.Timestamp.dt.hour
year1_consumption

,Timestamp,Load_LRPlugLoadsPowerUsage,Load_KPlugLoadsPowerUsage,Load_MBRPlugLoadsPowerUsage,Load_BR2PlugLoadsPowerUsage,Load_BR3PlugLoadsPowerUsage,Load_BR4PlugLoadsPowerUsage,Load_1stFloorSensHeatPowerUsage,Load_2ndFloorSensHeatPowerUsage,Load_1stFloorLightsPowerUsage,...,Elec_PowerParentBDownstairs,Elec_PowerPlugsInstKitD,Elec_PowerPlugsDRB,Elec_PowerPlugsAttic,DHW_HeatPumpWaterHeaterPowerTotal,DHW_SHWPumpsPowerWithStandby,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HourOfTimestamp
0,2013-07-01 04:00:00+00:00,61.828361,0.82,0.000000,16.717136,21.385995,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,6.920000,3.36,95.285085,710.005600,NaN,4
1,2013-07-01 05:00:00+00:00,61.741718,0.82,0.000000,16.720044,21.356809,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,6.920000,3.36,56.473583,360.019787,NaN,5
2,2013-07-01 06:00:00+00:00,61.282300,0.82,0.000000,16.603185,21.266155,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,6.920000,3.36,88.228208,688.178560,NaN,6
3,2013-07-01 07:00:00+00:00,61.351621,0.82,0.000000,16.574143,21.199436,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,6.920000,3.36,50.757458,323.083507,NaN,7
4,2013-07-01 08:00:00+00:00,61.047820,0.82,0.000000,16.615978,21.224998,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,6.920000,3.36,50.538875,435.437653,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2014-07-01 00:00:00+00:00,30.754906,0.82,0.000000,53.912077,8.161264,86.011407,174.346985,104.856611,132.203343,...,73.312845,44.951712,0.0,0.0,6.920000,3.36,102.977250,822.883507,NaN,0
8757,2014-07-01 01:00:00+00:00,0.000000,0.82,0.000000,54.106752,0.000000,86.304487,141.672519,140.799554,136.240926,...,73.866524,45.050383,0.0,0.0,6.920000,3.36,130.118958,1295.618133,NaN,1
8758,2014-07-01 02:00:00+00:00,0.000000,0.82,28.743771,20.741190,0.000000,41.914702,33.966106,247.097007,68.259084,...,0.000000,45.482085,0.0,0.0,15.309650,3.36,81.914167,544.670640,NaN,2
8759,2014-07-01 03:00:00+00:00,0.000000,0.82,19.566401,20.586135,0.000000,0.000000,0.000000,269.158059,0.000000,...,0.000000,43.915393,0.0,0.0,693.102459,3.36,90.396591,812.768994,NaN,3


In [50]:
hourON = year1_consumption.groupby(["HourOfTimestamp"])[consumption].sum()
hourON.reset_index(inplace=True)
hourON

,HourOfTimestamp,Load_LRPlugLoadsPowerUsage,Load_KPlugLoadsPowerUsage,Load_MBRPlugLoadsPowerUsage,Load_BR2PlugLoadsPowerUsage,Load_BR3PlugLoadsPowerUsage,Load_BR4PlugLoadsPowerUsage,Load_1stFloorSensHeatPowerUsage,Load_2ndFloorSensHeatPowerUsage,Load_1stFloorLightsPowerUsage,...,Elec_PowerParentBUpstairs,Elec_PowerParentBDownstairs,Elec_PowerPlugsInstKitD,Elec_PowerPlugsDRB,Elec_PowerPlugsAttic,DHW_HeatPumpWaterHeaterPowerTotal,DHW_SHWPumpsPowerWithStandby,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower
0,0,21940.235618,822.814934,0.537771,18633.069216,6555.556573,25920.659085,65828.610295,33355.141849,44803.179444,...,65.350087,26123.011094,15468.455299,7.977786,0.000000,58982.099820,1763.530711,66477.913587,174814.401666,0.0
1,1,17480.300893,295.629232,144.461384,17164.280831,1561.201126,22726.147086,65188.342555,36617.614156,46189.482026,...,192.176874,26086.022289,15468.658266,7.607601,0.000000,77609.394374,1439.599285,61395.103717,167218.776418,0.0
2,2,13076.534479,295.685035,8717.376665,10963.608389,194.802726,14447.210687,43343.564576,58731.198061,37019.611403,...,11590.811686,14138.542327,15558.478896,7.595192,12.369004,78521.880721,1257.900124,78305.919292,183019.939923,0.0
3,3,8514.117035,295.443000,11055.159716,7869.829839,193.921351,2372.816531,15770.216367,83878.681703,16062.668143,...,19721.316648,5032.674880,15453.503853,7.314828,3.778955,60749.284990,1230.240444,74544.208573,168311.296689,0.0
4,4,5950.387364,296.207072,3682.599989,7877.820664,214.776143,87.288489,2984.643316,92169.258016,2866.457858,...,22181.476789,1378.944132,14963.603307,7.298610,0.000000,15487.045227,1232.969451,91028.042189,185202.545252,0.0
5,5,5350.144344,306.620585,296.542408,7885.994300,221.999596,513.462292,466.954726,96797.019413,334.170827,...,23873.999887,120.831126,15263.578829,7.159510,0.000000,9244.310160,1233.874133,129509.211053,194725.168039,0.0
6,6,5222.841724,339.392696,216.216712,7903.450140,271.253096,627.827365,152.637987,98868.473131,389.323378,...,24318.786398,82.588718,15584.158837,7.261024,0.000000,6937.862925,1229.173228,92950.535561,196899.036783,0.0
7,7,5226.352372,299.171878,0.000000,7885.120820,256.104446,63.741809,303.222145,98691.244063,527.685088,...,24283.030571,135.439817,15565.324689,7.275294,0.000000,9960.032587,1229.158026,101565.249667,210512.451905,0.0
8,8,5255.894194,295.200000,0.000000,7865.121097,215.029724,0.000000,363.082836,98644.998909,531.899685,...,24300.611111,134.911797,15543.729437,7.288610,0.000000,11332.975248,1229.160301,123395.423720,219048.204744,0.0
9,9,5251.991263,295.200000,21.596914,7861.807371,213.599968,0.000000,503.480685,98606.679872,732.393869,...,24240.536245,239.286019,15517.717742,7.298934,0.000000,20694.685307,1229.154215,88628.781408,221378.111484,0.0


In [51]:
def normalise_dataframe(df):
    '''
    Normalises values of all attributes in a dataframe, with minmax method.
    :param df:
    :type df:
    :return:
    :rtype:
    '''
    normalised_df = df.copy()

    for column in normalised_df:
        try:
            normalised_df[column] = (normalised_df[column] - normalised_df[column].min()) / (
                    normalised_df[column].max() - normalised_df[column].min())
        except:
            normalised_df[column] = normalised_df[column]

    return normalised_df

In [52]:
sum_hourON = hourON.sum()
hourON = hourON[sum_hourON.sort_values(ascending=True).index]
sum_gt_zero_list = hourON.sum(0).loc[hourON.sum(0) > 0].index.tolist()
hourON=hourON[sum_gt_zero_list]
hourON

,Elec_PowerPlugsInstBR4B,Elec_PowerPlugsInstBA2,Elec_PowerPlugsInstMudroomA,Elec_PowerPlugsAttic,Elec_PowerPlugsInstBR4A,Elec_PowerPlugsBA2North,Elec_PowerPlugsInstLRA,Elec_PowerPlugsInstMBAB,HourOfTimestamp,Elec_PowerLightsAttic,...,Load_RefrigeratorPowerWithStandby,Load_BasementPlugLoadsPowerUsage,Elec_PowerHRV,Load_DryerPowerTotal,Elec_PowerHeatPumpWH2of2,Load_1stFloorSensHeatPowerUsage,Load_2ndFloorSensHeatPowerUsage,DHW_HeatPumpWaterHeaterPowerTotal,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower
0,0.010067,0.875876,0.559791,0.000000,0.000000,0.006898,0.037146,30.874961,0,56.975516,...,17491.617404,21512.988439,21686.477152,65865.372044,28491.148061,65828.610295,33355.141849,58982.099820,66477.913587,174814.401666
1,0.030413,0.946481,0.613814,0.000000,0.000000,0.000000,0.114007,0.485846,1,97.724514,...,17141.793293,21494.207050,21582.089140,39927.814582,37851.901153,65188.342555,36617.614156,77609.394374,61395.103717,167218.776418
2,0.020125,0.455079,0.587549,12.369004,0.000000,0.002321,0.076034,0.474559,2,89.940747,...,17152.129923,21526.752414,21600.702172,3209.414534,38350.733603,43343.564576,58731.198061,78521.880721,78305.919292,183019.939923
3,0.000000,0.000000,0.511551,3.778955,0.000000,0.000000,0.000000,0.481727,3,166.942601,...,17104.136341,21504.320770,21491.244466,582.503438,29389.308511,15770.216367,83878.681703,60749.284990,74544.208573,168311.296689
4,0.157086,0.158072,0.570218,0.000000,0.000000,0.000000,0.595163,0.000000,4,4.150276,...,17008.501499,21334.784835,21672.381847,0.000000,6622.153888,2984.643316,92169.258016,15487.045227,91028.042189,185202.545252
5,0.052366,0.052580,0.189596,0.000000,0.000000,0.000000,0.197812,0.000000,5,0.000000,...,17141.294012,21365.333808,21673.264899,0.000000,3421.950894,466.954726,96797.019413,9244.310160,129509.211053,194725.168039
6,0.022596,0.022802,0.082221,0.000000,0.000000,0.000000,0.085778,0.000000,6,0.000000,...,17266.935585,21442.578616,21525.426833,0.000000,2245.302463,152.637987,98868.473131,6937.862925,92950.535561,196899.036783
7,0.010058,0.010111,0.036372,0.000000,0.000000,0.000000,0.037954,0.000000,7,0.000000,...,17235.131078,21422.551670,21580.192835,0.000000,3769.189294,303.222145,98691.244063,9960.032587,101565.249667,210512.451905
8,0.010025,0.010101,0.036385,0.000000,0.000000,0.000000,0.038001,0.000000,8,0.000000,...,16905.067804,21368.788325,21613.578553,0.000000,4463.964624,363.082836,98644.998909,11332.975248,123395.423720,219048.204744
9,0.010325,0.010305,0.037283,0.000000,0.000000,0.000000,0.038482,0.000000,9,0.000000,...,16355.719949,21341.249603,21633.293097,0.000000,9183.167987,503.480685,98606.679872,20694.685307,88628.781408,221378.111484


In [53]:
import plotly.graph_objects as go

#set up data to heatmap
columns = hourON.columns[1:].tolist()
# hourON[columns] = normalise_dataframe(hourON[columns])

z_data = []
for i in hourON[columns]:
    data_list = hourON[i].round(2).values.tolist()
    z_data.append(data_list)

# #Annotate z_values
# annotations = go.Annotations()
# for n, row in enumerate(z_data):
#     for m, val in enumerate(row):
#         annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
#                                          x=hourON["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
#                                          showarrow=False))

viridis = px.colors.sequential.Magma

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON["HourOfTimestamp"],
    y=columns,
    # colorscale='Viridis'
    colorscale = [
        [0, viridis[0]],
        [1./1000000, viridis[2]],
        [1./10000, viridis[4]],
        [1./100, viridis[7]],
        [1., viridis[9]],
    ],
    colorbar= dict(
        tick0= 0,
        tickmode= 'array',
        tickvals= [0, 100000, 200000]
    )
)
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    # annotations=annotations,
)

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)

# fig.write_html(Db.get_save_file_directory("hour_of_day_energy_consumption.html"))
#
# fig.show()

In [54]:
fig = px.imshow(hourON[columns].T)


fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    # annotations=annotations,
)

fig.show()